In [61]:
import boto3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sagemaker.session import Session
from joblib import dump, load

from sklearn.feature_extraction.text import HashingVectorizer

In [62]:
assetMDL = load('Outs/asset_svc_mdl_v1.joblib')
liableMDL = load('Outs/liability_svc_mdl_v1.joblib')

In [63]:
# load in asset and liability dataframes
assetDF = pd.read_csv('unstructAsset.csv')
liableDF = pd.read_csv('unstructLiable.csv')

In [65]:
# journal of physics A, Journal Stat. Physics
def structured_data(unstructured_df:pd.DataFrame, cluster_df:pd.DataFrame) -> pd.DataFrame:
    """
    Constructs a structured dataset from an unstructured column set
    
    :param: unstructured_df (type pandas.DataFrame)
        unstuructured pandas dataframe with loose column construction 
    :param: cluster_df (type pandas.DataFrame)
        a pandas dataframe of clustered labels and corresponding line items
    :param: (type numpy array)
        all corresponding cluster labels cirresponding with 'cluster_df' parameter
        
    :return: (type pandas DataFrame)
    """
    
    structured_df = pd.DataFrame()
    label_names = np.unique(cluster_df.Labels.values)
    remap = {}
    
    # assume that the there exists columns 'CIK' and 'Year' for unstructured data
    structured_df = unstructured_df[['CIK', 'Name', 'Year']]
    
    for label in label_names:
        data = cluster_df[cluster_df['Labels'] == label]['LineItems']     # filter by corresponding cluster
        
        # we first select all predicted columns, then sum across rows for only numeric figures
        selection = unstructured_df[data.values]
        
        sumV = selection.sum(axis=1, numeric_only=True)
        
        # we then select rows from the original unstructured dataframe with only np.nan and convert sumV index to np.nan
        # handle for Missing (NaN) and blank terms (0.0)
        sumV[selection.isnull().all(axis=1)] = np.nan
        
        # assign dictionary to have labels and matching vector
        remap[label] = sumV

    structured_df = structured_df.assign(**remap)   
    return structured_df

In [66]:
def company_pdf(df:pd.DataFrame, mdl):
    """
    Return a dataframe for a company showcasing its column names, the predicted class and the original values
    """
    
    # split values for company dataframe according to columns and values
    colNames = df.index
    colValues = df.values
    
    # predicting the column groups
    predNames = mdl.predict(HashingVectorizer(n_features=1000).fit_transform(colNames))
    
    retDF = pd.DataFrame({'Original Lineitems': colNames, 'Predicted Lineitems': predNames, 'Line values': colValues})
    
    return retDF

## Use Classificaiton model to predict label names for each line item

In [67]:
asset_predictions = pd.DataFrame([assetDF.columns[3:], 
                                  assetMDL.predict(HashingVectorizer(n_features=1000).fit_transform(assetDF.columns[3:]))], 
                                 index=['LineItems', 'Labels']).T

liable_predictions = pd.DataFrame([liableDF.columns[3:], 
                                   liableMDL.predict(HashingVectorizer(n_features=1000).fit_transform(liableDF.columns[3:]))], 
                                  index=['LineItems', 'Labels']).T

### Structured Asset Terms

In [69]:
# construct the strucutred data set 
tempdf = structured_data(assetDF, asset_predictions)
tempdf.to_csv('structAsset.csv', index=False)

In [89]:
assetDF[(assetDF.CIK == 68136) & (assetDF.Year == 2005)].iloc[0].iloc[3:].dropna().values

array([19623740.0, 28089836.0, 225931.0, 28628319.0, 22286699.0,
       14987555.0, 91272451.0, 14937286.0, 650055.0, 2037285.0, 460502.0,
       101670.0, 195131438.0, 67497558.0, 37568640.0, 24354707.0,
       311092.0], dtype=object)

In [76]:
company_pdf(assetDF[(assetDF.CIK == 68136) & (assetDF.Year == 2005)].iloc[0].iloc[3:].dropna(), assetMDL)

,Original Lineitems,Predicted Lineitems,Line values
0,Affiliates,Other assets,1.96237e+07
1,"Brokers, dealers and clearing organizations",Receivable from broker-dealers,2.80898e+07
2,Cash,Cash and cash equivalents,225931
3,Cash and securities deposited with clearing or...,Cash and cash equivalents,2.86283e+07
4,Corporate and other debt,Other assets,2.22867e+07
5,Corporate equities,Other assets,1.49876e+07
6,Customers,Receivables from customers and counterparties,9.12725e+07
7,Derivative contracts,Other assets,1.49373e+07
8,Fees and other,Other assets,650055
9,Interest and dividends,Other assets,2.03728e+06


### Structured Liability Terms

In [37]:
# construct the strucutred data set 
tempdf = structured_data(liableDF, liable_predictions)
tempdf.to_csv('structLiable.csv', index=False)

In [38]:
tempdf

,CIK,Name,Year,Accounts payable,Accrued liabilities,Additional Paid-in capital,"Common stock, par value",Current liabilities,Government and agency securities obligations,Income tax payable,...,Repurchase Agreements (repo),Retained (Accumulated) earnings,Securities borrowed,Securities sold short,Short-term borrowing,Subordinated liabilities,Total liabilities,Total liabilities and shareholder's equity,Total shareholder's equity,Treasury stock
0,1224385,"WELLS FARGO SECURITIES, LLC",2004,NaN,27595.0,NaN,NaN,NaN,NaN,NaN,...,13602472.0,39849.0,NaN,3686980.0,NaN,538500.0,NaN,NaN,NaN,NaN
1,1224385,"WELLS FARGO SECURITIES, LLC",2005,NaN,83799.0,NaN,NaN,NaN,NaN,NaN,...,21430507.0,NaN,NaN,6775259.0,NaN,1183500.0,NaN,NaN,2107707.0,NaN
2,1224385,"WELLS FARGO SECURITIES, LLC",2006,NaN,59321.0,NaN,NaN,NaN,NaN,NaN,...,12026974.0,NaN,NaN,5059406.0,NaN,1183500.0,NaN,NaN,2727933.0,NaN
3,1224385,"WELLS FARGO SECURITIES, LLC",2007,2536.0,75497.0,NaN,NaN,NaN,NaN,NaN,...,9294056.0,902450.0,NaN,4241568.0,NaN,1183500.0,NaN,NaN,NaN,NaN
4,1224385,"WELLS FARGO SECURITIES, LLC",2008,NaN,92477.0,NaN,NaN,NaN,NaN,NaN,...,11870678.0,724933.0,NaN,3605979.0,NaN,1333500.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,91154,CITIGROUP GLOBAL MARKETS INC.,2016,NaN,NaN,9099.0,10.0,NaN,24756.0,NaN,...,129960.0,33.0,NaN,NaN,10606.0,9945.0,NaN,NaN,11273.0,NaN
155,91154,CITIGROUP GLOBAL MARKETS INC.,2017,NaN,NaN,8921.0,10.0,NaN,36821.0,NaN,...,138098.0,263.0,NaN,NaN,5576.0,9945.0,NaN,NaN,11873.0,NaN
156,91154,CITIGROUP GLOBAL MARKETS INC.,2018,NaN,NaN,8937.0,10.0,NaN,31492.0,NaN,...,146794.0,68.0,NaN,NaN,1251.0,9945.0,NaN,NaN,9969.0,NaN
157,91154,CITIGROUP GLOBAL MARKETS INC.,2019,NaN,NaN,8802.0,10.0,NaN,37088.0,NaN,...,157690.0,48.0,NaN,NaN,508.0,9945.0,NaN,NaN,18337.0,NaN
